In [ ]:
# 定义一个吸引度公式

# 定义一个矩阵（excel），每一行代表一个会员，每一列代表一个任务
# 会员数据来源于../data/有效会员信息数据.xlsx中的会员编号列
# 任务数据来源于../data/有效已结束项目任务位置数据.xlsx中的任务号码列

# 吸引度的公式为：$$\sqrt{\frac{a}{distance^2}+b*price^2}$$
# 其中a和b为参数，distance为会员到任务的距离（出自会员数据中的经度和纬度列以及任务数据中的任务gps 纬度和任务gps经度列），price为任务的价格（出自任务数据中的任务标价列）

# 请根据上述公式，计算出每一个会员对每一个任务的吸引度，并将结果保存在一个新的矩阵中

In [2]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Load data
members_df = pd.read_excel('../data/有效会员信息数据.xlsx')
tasks_df = pd.read_excel('../data/有效已结束项目任务位置数据.xlsx')

# Extract relevant columns
member_ids = members_df['会员编号']
task_ids = tasks_df['任务号码']
member_coords = members_df[['经度', '纬度']].values
task_coords = tasks_df[['任务gps经度', '任务gps 纬度']].values
task_prices = tasks_df['任务标价'].values

# Convert longitude from 0-180 to -90-90
def convert_longitude(lon):
    return lon - 90

member_coords[:, 0] = np.vectorize(convert_longitude)(member_coords[:, 0])
task_coords[:, 0] = np.vectorize(convert_longitude)(task_coords[:, 0])

# Parameters
a = 0.01  # Example value, adjust as needed
b = 0.0001  # Example value, adjust as needed

# Initialize the attractiveness matrix
attractiveness_matrix = np.zeros((len(member_ids), len(task_ids)))

# Calculate attractiveness
for i, member_coord in enumerate(member_coords):
    for j, task_coord in enumerate(task_coords):
        distance = geodesic(member_coord, task_coord).kilometers
        price = task_prices[j]
        attractiveness = np.sqrt(a / (distance ** 2) + b * (price ** 2))
        attractiveness_matrix[i, j] = attractiveness

# Convert to DataFrame for better readability
attractiveness_df = pd.DataFrame(attractiveness_matrix, index=member_ids, columns=task_ids)

# Save the result to a new Excel file
attractiveness_df.to_excel('../data/会员任务吸引度矩阵.xlsx')